## Importing the libraries

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

## Importing the dataset

In [20]:
dataset = pd.read_excel("dataset.xlsx")
dataset.head()

,KOD,BARKOD,MAMUL,K.İÇİ,GR,LİSTE,FATURA ALTI %,DÖNEM SONU %,KDV,NET MALİYET,Ö.RAF,KATEGORİ,AY
0,67293883,8690637840746,1,8,400.0,21.85,12.0,5,0.01,18.449266,23.061583,FOOD,MART
1,67293891,8690637840821,1,8,400.0,21.85,12.0,5,0.01,18.449266,23.061583,FOOD,MART
2,67767533,8690637905896,1,12,600.0,27.00,13.0,5,0.01,22.538655,28.173319,FOOD,MART
3,67754288,8690637905193,1,12,600.0,27.00,13.0,5,0.01,22.538655,28.173319,FOOD,MART
4,68612788,8690637999277,1,8,335.0,39.75,10.0,5,0.01,34.326113,42.907641,FOOD,MART


In [21]:
selected_columns = ['KATEGORİ', 'AY', 'MAMUL', 'K.İÇİ', 'GR', 'LİSTE', 'NET MALİYET']
data_selected = dataset[selected_columns]
data_selected.head()

,KATEGORİ,AY,MAMUL,K.İÇİ,GR,LİSTE,NET MALİYET
0,FOOD,MART,1,8,400.0,21.85,18.449266
1,FOOD,MART,1,8,400.0,21.85,18.449266
2,FOOD,MART,1,12,600.0,27.00,22.538655
3,FOOD,MART,1,12,600.0,27.00,22.538655
4,FOOD,MART,1,8,335.0,39.75,34.326113


In [22]:
categorical_columns = ['AY', 'MAMUL']
numeric_columns = ['K.İÇİ', 'GR', 'LİSTE']

In [23]:
X = data_selected.drop(columns=["KATEGORİ"])  # Giriş verileri
y = data_selected["KATEGORİ"]  # Çıkış (etiket) verileri

In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

# Sayısal ve kategorik sütunları işlemek için bir sütun dönüştürücü oluşturun
preprocessor = ColumnTransformer(
    transformers=[
        ("sayısal", StandardScaler(), numeric_columns),
        ("kategori", OneHotEncoder(), categorical_columns),
    ],
    remainder="passthrough",
)

le = LabelEncoder()

# Verileri dönüştürün
X = preprocessor.fit_transform(X)
y = le.fit_transform(y)


In [25]:
x_dataset = pd.DataFrame(X.toarray())
x_dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-0.565928,-0.110449,-0.986225,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.449266
1,-0.565928,-0.110449,-0.986225,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.449266
2,-0.470838,0.121934,-0.829472,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.538655
3,-0.470838,0.121934,-0.829472,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.538655
4,-0.565928,-0.185974,-0.441394,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.326113


In [26]:
from sklearn.model_selection import train_test_split

# Verileri eğitim ve test kümelerine bölün
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Modeli oluşturun

In [27]:
from tensorflow import keras
from keras import layers

model = keras.Sequential([ # Sıralı bir model oluşturun
    layers.Dense(64, activation='relu', input_shape=[X_train.shape[1]]), # Giriş katmanı
    layers.Dense(64, activation='relu'), # Gizli katman
    layers.Dense(3, activation='softmax') # Çıkış katmanı
])


#### Modeli Derle

In [28]:
model.compile(
    optimizer='adam', # Optimizasyon fonksiyonu
    loss='sparse_categorical_crossentropy', # Kayıp fonksiyonu
    metrics=['accuracy'] # Başarı metriği
)

#### Eğitim işlemini başlatın

In [30]:
X_train = X_train.toarray()
y_train = y_train.reshape(-1, 1)
history = model.fit(
    X_train, y_train, # Eğitim verileri
    validation_split=0.2, # Eğitim verilerinin %20'sini doğrulama verileri olarak kullanın
    batch_size=32, # Her iterasyonda 32 örnek kullanın
    epochs=100, # 100 iterasyon
    verbose=1 # Her iterasyonda ekrana çıktı ver
)

Epoch 1/100


77/77 [==============================] - 1s 2ms/step - loss: 1.0500 - accuracy: 0.5368 - val_loss: 0.7869 - val_accuracy: 0.6356
Epoch 2/100
77/77 [==============================] - 0s 921us/step - loss: 0.5196 - accuracy: 0.8429 - val_loss: 0.3795 - val_accuracy: 0.8791
Epoch 3/100
77/77 [==============================] - 0s 908us/step - loss: 0.2303 - accuracy: 0.9550 - val_loss: 0.1488 - val_accuracy: 0.9984
Epoch 4/100
77/77 [==============================] - 0s 912us/step - loss: 0.0815 - accuracy: 0.9984 - val_loss: 0.0614 - val_accuracy: 1.0000
Epoch 5/100
77/77 [==============================] - 0s 1ms/step - loss: 0.0418 - accuracy: 0.9992 - val_loss: 0.0337 - val_accuracy: 1.0000
Epoch 6/100
77/77 [==============================] - 0s 1ms/step - loss: 0.0226 - accuracy: 1.0000 - val_loss: 0.0191 - val_accuracy: 1.0000
Epoch 7/100
77/77 [==============================] - 0s 961us/step - loss: 0.0131 - accuracy: 1.0000 - val_loss: 0.0140 - val_accuracy: 1.0000
Epoch 8/100
77/77

#### Eğitim ve doğrulama kayıp grafiği

In [31]:
# Eğitim ve doğrulama kayıp grafiği ve Eğitim ve doğrulama başarı grafiği

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(y=history.history['loss'], name="Eğitim Kaybı"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=history.history['val_loss'], name="Doğrulama Kaybı"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=history.history['accuracy'], name="Eğitim Başarısı"),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(y=history.history['val_accuracy'], name="Doğrulama Başarısı"),
    row=1, col=2
)

fig.update_layout(height=500, width=800, title_text="Eğitim ve Doğrulama Metrikleri")
fig.show()



In [32]:
# Test verileri üzerinde modeli değerlendirin
X_test = X_test.toarray()
y_test = y_test.reshape(-1, 1)
model.evaluate(X_test, y_test, verbose=2)

# Test verileri üzerinde tahmin yapın
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred = le.inverse_transform(y_pred)

y_test = le.inverse_transform(y_test)

from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

cm = confusion_matrix(y_test, y_pred, labels=["Food", "HC", "PC"])

fig = ff.create_annotated_heatmap(
    z=cm,
    x=["Food", "HC", "PC"],
    y=["Food", "HC", "PC"],
    colorscale='Viridis'
)

fig.update_layout(title_text='Karmaşıklık Matrisi', xaxis = dict(title='Tahmin'), yaxis = dict(title='Gerçek'))

fig.show()


24/24 - 0s - loss: 3.9242e-07 - accuracy: 1.0000 - 28ms/epoch - 1ms/step
24/24 [==============================] - 0s 522us/step


c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:155: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

